In [1]:
#add spex templates to selection function
#select the highest SNR spectra of each spectral type i.e 

In [2]:
import splat
import wisps
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
import numba
%matplotlib inline

Adding 145 sources from /Users/caganze/research/splat//resources/Spectra/Public/LRIS-RED/ to spectral database
Adding 89 sources from /Users/caganze/research/splat//resources/Spectra/Public/MAGE/ to spectral database
Adding 2404 sources from /Users/caganze/research/splat//resources/Spectra/Public/SPEX-PRISM/ to spectral database


In [4]:
#pectra=pd.read_pickle(wisps.OUTPUT_FILES+'/l_t_dwarfs_spex.pkl.gz')

In [ ]:
s=pectra[3].splat_spectrum
s.toInstrument('WFC3-G141')
s.dof

In [ ]:
spe=splat.getSpectrum(lucky=True)[0]

In [ ]:
spe.dof

In [ ]:
113-2

In [ ]:
from scipy import stats

In [ ]:
data=wisps.datasets['spex']

In [ ]:
x=np.sort((data.spex_chi/data.line_chi).values)

In [ ]:
x

In [ ]:
plt.plot(x, stats.f.cdf(x, 518, 111, x.mean(), np.ptp(x)), lw=5, alpha=0.6, label='scalled')
plt.plot(x, stats.f.cdf(x, 518, 111), lw=5, alpha=0.6, label='unscalled')
plt.axvline(1)
plt.legend()

In [ ]:
#dofs=[x.splat_spectrum.toInstrument('WFC3-G141').dof for x in pectra ]

In [ ]:
splat.initializeStandards()

In [ ]:
spts=np.array([splat.typeToNum(x) for x in np.arange(16, 42)])

In [ ]:
#remove object from forbidden list
splat_db0=splat.searchLibrary(vlm=True, giant=False, young=False)
splat_db0['SHORTNAME']=splat_db0.DESIGNATION.apply(splat.designationToShortName)
splat_db=splat_db0[~splat_db0.SHORTNAME.isin(wisps.FORBIDDEN_LIST)].reset_index(drop=True)

In [ ]:
df=splat_db[splat_db.SPEX_TYPE.isin(spts)]

In [ ]:
splat_within_snr=df[df.MEDIAN_SNR.between(5, 200)]

In [ ]:
import splat.database as splatdb

In [ ]:
#binary vetting
folder='/users/caganze/research/wisps/data/'
bindf=pd.read_csv(folder+'/binaries.txt', names=['name', ' '], sep=',')

In [ ]:
groups=splat_within_snr.sort_values('MEDIAN_SNR',ascending=False).groupby('SPEX_TYPE')

In [ ]:
picks=[]
for g in groups:
    picks.append(g[1].nlargest(20,'MEDIAN_SNR'  ))

In [ ]:
df_picks=pd.concat(picks)

In [ ]:

def create_splat_source(filename):
    try:
        #create a wisps spectrum
        #retrieve splat spectrum

        splat_s=splat.getSpectrum(filename=filename)[0]
        #spt=splat.typeToNum(splat_s.spex_type)
        splat_s.toInstrument('WFC3-G141')

        #create a source object
        s=wisps.Spectrum( 
                       wave=splat_s.wave.value, 
                       flux=splat_s.flux.value,
                      noise=splat_s.noise.value)
        
        #put it on wisps resolution
        
        s._splat_spectrum=splat_s

        s.normalize()
        return s
    except :
        return 

In [ ]:
import wisps

In [ ]:
df_picks['spt']=df_picks.SPEX_TYPE.apply(wisps.make_spt_number)

In [ ]:
df_picks=df_picks.sort_values(by='spt')

In [ ]:
#df_picks

In [ ]:
%%capture
spectra=df_picks.DATA_FILE.apply(create_splat_source)

In [ ]:
def plot_sp_sp(s, a, shortname):
    std=splat.STDS_DWARF_SPEX[splat.typeToNum(s.spectral_type[0])]
    std.normalize(waverange=[1.1, 1.7])
    mask2=np.logical_and(std.wave.value>0.8, std.wave.value<2.5)
    
    mask=np.logical_and(s.wave>0.8, s.wave<2.5)
    a.plot(s.wave[mask], s.flux[mask], label=shortname,linewidth=3)
    a.plot(std.wave.value[mask2], std.flux.value[mask2], linestyle='--', label='std', alpha=0.5)
    
    
    a.set_title("{} ".format(s.spectral_type ))
    a.legend()


In [ ]:
(spectra.iloc[0])

In [ ]:
#for idx, s in enumerate(spectra.dropna().values):
#    s.normalize()
#    fig, ax=plt.subplots()
#    plot_sp_sp(s, ax, idx)
    
    
   

In [ ]:
def make_wisps(splat_s):
    try:
        s=wisps.Spectrum(wave=splat_s.wave.value, 
                           flux=splat_s.flux.value,
                          noise=splat_s.noise.value)
        return s
    except:
        return 

In [ ]:
ydwarfs_stds=pd.read_pickle('/users/caganze/y_standards.pkl')
ydwarfs_data=pd.read_pickle(wisps.OUTPUT_FILES+'/schneider_spectra.pkl.gz')

In [ ]:
t9s=[make_wisps(x) for x in splat.getSpectrum(spt='T9.0')]
t8s=[make_wisps(x) for x in splat.getSpectrum(spt='T8.0')]
y0=[make_wisps(x ) for x in ydwarfs_stds.spec]
y1=[make_wisps(x) for x in ydwarfs_stds.spec]
#l3s=[make_wisps(x) for x in splat.getSpectrum(spt='L3.0', binary=False, vlm=True, young=False)]

In [ ]:
extras=[x for x in np.concatenate([t9s, t8s, y0, y1])]
extras=[x for x in extras if x]

In [ ]:
retain=np.append(spectra.dropna().values, extras)
retain=np.append(retain, ydwarfs_data)

In [ ]:
dfsp=pd.DataFrame()
dfsp['spectra']=retain
dfsp['spt']= [x.spectral_type for x in retain]

In [ ]:
import pickle
with open(wisps.OUTPUT_FILES+'/l_t_dwarfs_spex.pkl', 'wb') as file:
    pickle.dump(retain,file)

In [ ]:
hgjbknl

In [ ]:
rdpck=pd.read_pickle(wisps.OUTPUT_FILES+'/l_t_dwarfs_spex.pkl')

In [ ]:
bindf['shortname']=bindf.name.apply(str).apply(splat.designationToShortName)

In [ ]:
splat_within_snr['shortname']=splat_within_snr.DESIGNATION.apply(splat.designationToShortName)

In [ ]:
bsample=splat_within_snr[ ~ (splat_within_snr.shortname.isin(bindf.shortname) |
                             np.isnan(splat_db['PARALLAX']) | (splat_within_snr['SBINARY'] =='Y'))]

In [ ]:
bsample=bsample.reset_index(drop=True)

In [ ]:
%%capture
bsample_spectra=bsample.DATA_FILE.apply(create_splat_source)

In [ ]:
bsample['spectra']=bsample_spectra

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
bsample=bsample.drop_duplicates(subset='shortname').reset_index(drop=True)

In [ ]:
dfextr=pd.DataFrame()
#dfextr['NAME']=['J045853.90+643451.9', 'WISE1738+2732', 'WISE1738+2732']
dfextr['SPEX_TYPE']=[x.spectral_type for x in extras]
#dfextr['shortname']=['J045853.90+643451.9', 'WISE1738+2732', 'WISE1738+2732']
dfextr['spectra']=extras


In [ ]:
#add T9 and y dwarfs
#dfextr
bspl=bsample.append(dfextr, ignore_index=True, verify_integrity=True, sort=True)

In [ ]:
bspl=bspl[~bspl.spectra.isna()]

In [ ]:
%%capture
with PdfPages(wisps.OUTPUT_FIGURES+'/multipage_binaries.pdf') as pdf:
    
    for g in np.array_split(bspl, int(len(bspl)/4)):
        fig, ax=plt.subplots(ncols=2, nrows=2)
        
        plot_sp_sp(g.spectra.iloc[0], ax[0][0], g.shortname.iloc[0])
        plot_sp_sp(g.spectra.iloc[1], ax[0][1],  g.shortname.iloc[1])
        plot_sp_sp(g.spectra.iloc[2], ax[1][0],  g.shortname.iloc[2])
        plot_sp_sp(g.spectra.iloc[3], ax[1][1],  g.shortname.iloc[3])

        pdf.savefig() 
        plt.close()

In [ ]:
bspl.reset_index(drop=True).to_pickle(wisps.OUTPUT_FILES+'/binary_spex.pkl')

In [ ]:
(bspl[['SOURCE_KEY', 'NAME', 'DESIGNATION', 'RA', 'DEC', 'DISCOVERY_REFERENCE',
       'OPT_TYPE', 'NIR_TYPE', 'LIT_TYPE',
       'J_2MASS',
       'H_2MASS_E', 'GRAVITY_CLASS_NIR','BINARY', 'SBINARY','SIMBAD_NAME', 'SIMBAD_SPT',
       'SIMBAD_SPT_REF', 'PARALLAX', 'PARALLAX_E', 'DISTANCE',
       'SPEX_GRAVITY_CLASSIFICATION', 
       'shortname']]).to_excel(wisps.OUTPUT_FILES+"/nonbinaries.xlsx")  